# Mining Massive Datasets: Problem Set 1
### Due October 28, 2019, 11:59 pm

Duc Anh, Phi 3550091

Mustafa, Ibrahim 3284705

Amrit, Sandhar

## Exercise 2
### b) Give three own programming examples of your choice for transformations (but not for map() or filter()) and three examples for actions (again, of your choice).

In [1]:
import findspark
findspark.init("/usr/local/spark")

In [68]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType, LongType

In [57]:
spark = SparkSession.builder.master('local').appName('lab1').getOrCreate()
sc = spark.sparkContext

In [58]:
nums = sc.parallelize([9,1,2,1,4,5,0,1,1,1,2,8,9])

# 3 example transformations
numsSortByKey = nums \
    .map(lambda num: (num, 1)) \
    .sortByKey()

numsReduceByKey = nums \
    .map(lambda num: (num, 1)) \
    .reduceByKey(lambda accum, val: accum+val)

numsGroupByKey = nums \
    .map(lambda num: (num,1 )) \
    .groupByKey()

# 3 example actions
nums.collect()
nums.take(2)
nums.count()


13

## Exercise 4

### a)

In [69]:
adultsRDD = sc.textFile("adult.data") \
    .map(lambda line: line.split(",")) \
    .filter(lambda p: len(p) == 15 ) \
    .map(lambda p: (
            p[3].strip(),
            p[4].strip(),
            p[5].strip(),
            p[9].strip(),
            p[12].strip(),
            p[13].strip(),
            p[14].strip()
    ))
#print(adultsRDD.first())
schema = StructType([
    StructField("education", StringType(), True),
    StructField("education_num", StringType(), True),
    StructField("marital_status", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("hours_per_week", StringType(), True),
    StructField("native_country", StringType(), True),
    StructField("income", StringType(), True)
])
dfAdults = spark.createDataFrame(adultsRDD, schema)
dfAdults.take(1)

[Row(education='Bachelors', education_num='13', marital_status='Never-married', sex='Male', hours_per_week='40', native_country='United-States', income='<=50K')]

### b)

In [70]:
maleMs = dfAdults.filter(dfAdults.sex=='Male').groupBy('marital_status').count().alias('maleMs')
maleMs = maleMs.withColumnRenamed('count', 'maleCount')
allMs = dfAdults.groupBy('marital_status').count().alias('allMs')
allMs = allMs.withColumnRenamed('count', 'totalCount')
joined = allMs.join(maleMs, allMs.marital_status == maleMs.marital_status, 'inner').drop(allMs.marital_status)
joined = joined.withColumn("maleRatio", joined['maleCount']/joined['totalCount'])
joined.show()

+----------+--------------------+---------+-------------------+
|totalCount|      marital_status|maleCount|          maleRatio|
+----------+--------------------+---------+-------------------+
|      1025|           Separated|      394|0.38439024390243903|
|     10683|       Never-married|     5916| 0.5537770289244595|
|       418|Married-spouse-ab...|      213| 0.5095693779904307|
|      4443|            Divorced|     1771|0.39860454647760524|
|       993|             Widowed|      168| 0.1691842900302115|
|        23|   Married-AF-spouse|        9|  0.391304347826087|
|     14976|  Married-civ-spouse|    13319| 0.8893563034188035|
+----------+--------------------+---------+-------------------+



### c)

In [73]:
femaleInc50k = dfAdults \
    .filter(dfAdults.sex=='Female') \
    .filter(dfAdults.income=='>50K')
femaleInc50k = femaleInc50k \
    .withColumn('hours_per_week', femaleInc50k['hours_per_week'].cast(IntegerType())) \
    .groupBy('native_country') \
    .agg({'hours_per_week': 'mean'})
femaleInc50k.show()

+--------------+-------------------+
|native_country|avg(hours_per_week)|
+--------------+-------------------+
|   Philippines| 40.083333333333336|
|       Germany|  36.57142857142857|
|        France|               45.0|
|        Greece|               65.0|
|        Taiwan|              36.75|
|     Nicaragua|               35.0|
|          Hong|               40.0|
|         India|               38.5|
|         China|               33.6|
|         Italy|               42.0|
|          Cuba|               23.0|
|         South| 56.666666666666664|
|          Iran|               40.0|
|       Ireland|               40.0|
|      Thailand|               50.0|
|          Laos|               40.0|
|   El-Salvador|               40.0|
|        Mexico|               35.0|
|      Honduras|               60.0|
|    Yugoslavia|               40.0|
+--------------+-------------------+
only showing top 20 rows



### d)

In [75]:
education_dict = {
    1 : "Preschool",
    2 : "1st-4th",
    3 : "5th-6th",
    4 : "7th-8th",
    5 : "9th",
    6 : "10th",
    7 : "11th",
    8 : "12th",
    9 : "HS-grad",
    10 : "-",
    11 : "-",
    12 : "-",
    13 : "-",
    14 : "-",
    15 : "-",
    16 : "Doctorate"
}
education_transform = udf(lambda x : education_dict[x], StringType())
minEd = dfAdults \
    .withColumn('education_num', dfAdults['education_num'].cast(IntegerType())) \
    .groupBy('income').min('education_num')
minEd = minEd.withColumn("min(education_num)", education_transform(minEd['min(education_num)']))
maxEd = dfAdults \
    .withColumn('education_num', dfAdults['education_num'].cast(IntegerType())) \
    .groupBy('income') \
    .max('education_num')
maxEd = maxEd.withColumn("max(education_num)", education_transform(maxEd['max(education_num)']))
joinedEd = minEd.join(maxEd, 'income', 'outer')
joinedEd.show()

+------+------------------+------------------+
|income|min(education_num)|max(education_num)|
+------+------------------+------------------+
| <=50K|         Preschool|         Doctorate|
|  >50K|           1st-4th|         Doctorate|
+------+------------------+------------------+



## Exercise 5
### a)

In [84]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [85]:
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", IntegerType(), True),
    StructField("education", StringType(), True),
    StructField("education-num", IntegerType(), True),
    StructField("marital-status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital-gain", IntegerType(), True),
    StructField("capital-loss", IntegerType(), True),
    StructField("hours-per-week", IntegerType(), True),
    StructField("native-country", StringType(), True),
    StructField("salary", StringType(), True)
])

train_df = spark.read.csv('train.csv', header=False, schema=schema)
test_df = spark.read.csv('test.csv', header=False, schema=schema)

In [86]:
categorical_variables = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]

encoder = OneHotEncoderEstimator(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

pipeline = Pipeline(stages=indexers + [encoder, assembler])

train_df = pipeline.fit(train_df).transform(train_df)

test_df = pipeline.fit(test_df).transform(test_df)

In [88]:
continuous_variables = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

assembler = VectorAssembler(
    inputCols=['categorical-features', *continuous_variables],
    outputCol='features'
)

train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

IllegalArgumentException: 'Output column features already exists.'

In [89]:
indexer = StringIndexer(inputCol='salary', outputCol='label')

train_df = indexer.fit(train_df).transform(train_df)

test_df = indexer.fit(test_df).transform(test_df)

train_df.limit(10).toPandas()['label']

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    1.0
8    1.0
9    1.0
Name: label, dtype: float64

In [107]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import numpy as np

gb = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

# Create ParamGrid for Cross Validation
gbParamGrid = (ParamGridBuilder()
             .addGrid(gb.maxDepth, np.arange(2, 5))
             .addGrid(gb.maxIter, [10])
             .build())

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

gbCv = CrossValidator(estimator=gb, estimatorParamMaps=gbParamGrid, evaluator=evaluator, numFolds=2)

gbCvModel = gbCv.fit(train_df)

In [108]:
from itertools import chain


transformed = gbCvModel.bestModel.transform(test_df)
attrs = sorted((attr["idx"], attr["name"]) for attr in (chain(*transformed.schema["features"].metadata["ml_attr"]["attrs"].values())))

In [109]:
gbCvFeatureImportance = pd.DataFrame([(name, gbCvModel.bestModel.featureImportances[idx]) for idx, name in attrs],columns=['feature_name','feature_importance'])

print(gbCvFeatureImportance.sort_values(by=['feature_importance'],ascending =False))

                                         feature_name  feature_importance
93                                                age            0.277161
23  categorical-features_marital-status-index-enco...            0.155221
98                                     hours-per-week            0.119090
95                                      education-num            0.106073
97                                       capital-loss            0.072674
..                                                ...                 ...
35  categorical-features_occupation-index-encoded_...            0.000000
33  categorical-features_occupation-index-encoded_...            0.000000
32  categorical-features_occupation-index-encoded_...            0.000000
30  categorical-features_occupation-index-encoded_...            0.000000
49      categorical-features_race-index-encoded_Black            0.000000

[99 rows x 2 columns]


### b)

In [111]:
gbCvModel.save('logistic_regression_model')
sameModel = CrossValidatorModel.load('logistic_regression_model')